In [49]:
import numpy as np
import cv2

# Option A: your original centroid‐arithmetic approach
# 1) Find centeriod 
# 2) Translate points so centeriod sit s on origin 
# 3) scale points around iriing 
# 4) Add what you intally subrtacted 
def scale_polygon_centroid(pts: np.ndarray, factor: float) -> np.ndarray:
    """
    Scale about the centroid by simple vector arithmetic.
    """
    c = pts.mean(axis=0)
    scaled = c + (pts - c) * factor
    return scaled.astype(np.int32)


# Option B: use OpenCV's getRotationMatrix2D + cv2.transform
def scale_polygon_cv2(pts: np.ndarray, factor: float) -> np.ndarray:
    """
    Build a 2×3 affine matrix that scales about the centroid and use cv2.transform.
    """
    # ensure float32 and shape (N,1,2)
    pts_cv = pts.reshape(-1, 1, 2).astype(np.int32)
    centroid = tuple(pts.mean(axis=0).tolist())
    # rotation=0, scale=factor
    M = cv2.getRotationMatrix2D(centroid, angle=0, scale=factor)
    scaled = cv2.transform(pts_cv, M)               # still shape (N,1,2)
    return scaled.reshape(-1, 2).astype(np.int32)


# Option C: Shapely’s built-in affinity.scale
from shapely.geometry import Polygon
from shapely.affinity  import scale as shapely_scale

def scale_polygon_shapely(pts: np.ndarray, factor: float) -> np.ndarray:
    """
    Wrap your points in a Shapely Polygon, scale by centroid, extract exterior coords.
    """
    poly = Polygon(pts)
    scaled_poly = shapely_scale(poly,
                                xfact=factor, yfact=factor,
                                origin='centroid')
    # .exterior.coords gives a closed ring (first==last), so drop the last:
    coords = np.array(scaled_poly.exterior.coords)[:-1]
    return coords.astype(np.int32)


# Option D: scikit-image's AffineTransform
from skimage.transform import AffineTransform

def scale_polygon_skimage(pts: np.ndarray, factor: float) -> np.ndarray:
    """
    Build an AffineTransform that scales about the centroid, then apply it.
    """
    c = pts.mean(axis=0)
    # translation = (1-factor)*centroid shifts things so that scaling is around c
    tf = AffineTransform(scale=(factor, factor),
                         translation=((1-factor)*c[0], (1-factor)*c[1]))
    scaled = tf(pts)   # returns float coords, shape (N,2)
    return scaled.astype(np.int32)


# Example of calling any of these:
court = np.array([[508,  65],
                  [796,  65],
                  [1175,570],
                  [210,570]], dtype=np.int32)
# court  = np.array([[506, 68], [798, 64], [1167, 564], [207, 569]])


In [50]:
def to_cv_contour(pts):
    return pts.reshape(-1,1,2)


buffer_small = scale_polygon_centroid(court, 0.9)
buffer_large = scale_polygon_centroid(court, 1.1)

court_cnt   = to_cv_contour(court)
small_cnt   = to_cv_contour(buffer_small)
large_cnt   = to_cv_contour(buffer_large)
# draw them
img = cv2.imread('../CourtDetection/Images/img_0005.jpg')


mask_large = np.zeros(img.shape[:2],dtype=np.int32)
mask_small = np.zeros_like(mask_large)

cv2.fillPoly(mask_large, [large_cnt], 255)
cv2.fillPoly(mask_small, [small_cnt], 255)
ring_mask = cv2.bitwise_and(mask_large, cv2.bitwise_not(mask_small))


# 5. make a colored overlay of same shape as img
overlay = img.copy()
overlay[ring_mask == 255] = (0, 0, 100)   # red ring

# 6. alpha-blend back onto original
alpha = 0.3
img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

# original court in yellow
img = cv2.polylines(img, [court_cnt], True, (0,255,255), 2)
# smaller buffer in green
cv2.polylines(img, [small_cnt], True, (0,255,0), 2)
# larger buffer in red
cv2.polylines(img, [large_cnt], True, (0,255,0), 2)

cv2.imshow('buffers', img)
cv2.waitKey(0)
cv2.destroyAllWindows()



